<a href="https://colab.research.google.com/github/ramkumardeepak774/Text_Classification_Flair-vs-Fasttext/blob/main/using_flair_to_classify_on_sysco_food.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install flair


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 30.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 99.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 70.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 78.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.9 MB/s eta 0:00:0

In [3]:
import pandas as pd
import numpy as np

In [5]:
from google.colab import files
sysco = files.upload()


Saving sysco-full.csv to sysco-full.csv


In [6]:
import io
data = pd.read_csv(io.BytesIO(sysco["sysco-full.csv"]))

In [7]:
data.head()

,Description,Request Type
0,Good afternoon. There was a delay in getting t...,General Inquiry
1,"Hello, We received the below deduction from yo...",Purchase Order
2,"Hi, Per attached Invoice, correct Pickup Allow...",Purchase Order
3,"Good afternoon, Sysco Florida South Co-op shor...",Purchase Order
4,"Good afternoon, Sysco Florida South Co-op shor...",Purchase Order


In [9]:
data.columns  = ['text', 'label']


In [22]:
data.shape

(23897, 2)

In [26]:
data = data.drop_duplicates()

In [44]:
data.dropna(inplace=True)
#drop empty cell column

<ipython-input-44-f1116dacf2bb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)


In [61]:
data = data.sample(frac = 0.17)

In [62]:
data

,text,label
6,"Hi, We have received a deduction for which we ...",General Inquiry
10657,HI Sabita The first two invoices (attached) po...,Purchase Order
6258,"Hi, Thank you for contacting us. Upon review, ...",Purchase Order
5687,"Hello customer service, We received a cash dis...",Purchase Order
9365,Attached is your invoice for PO 21709420 Pleas...,Purchase Order
...,...,...
22978,"Hi Team, Please request an invoice for open ag...",General Inquiry
5765,EXTERNAL EMAIL: This email originated from out...,General Inquiry
5520,Disregard sorry ______________________________...,General Inquiry
21371,Freight Andrea madrigal P | AP SMS Corporate D...,Purchase Order


In [63]:
from sklearn.model_selection import train_test_split


In [64]:
train_data, test_data = train_test_split(data, test_size=0.15, random_state=42)


In [65]:
mkdir -p data_fst


In [66]:
print(train_data.shape)
print(test_data.shape)


(2469, 2)
(436, 2)


In [67]:
train_data.to_csv("data_fst/train.csv")
test_data.to_csv("data_fst/test.csv")


In [15]:
import flair

In [16]:
from flair.datasets import ClassificationCorpus,CSVClassificationCorpus
from flair.data import Corpus

In [68]:
column_name_map = {2:"label_topic",1:"text"}


In [18]:
data_folder = '/content/data_fst/'


In [100]:
corpus_csv: Corpus = CSVClassificationCorpus(data_folder,column_name_map=column_name_map,skip_header=True, label_type='topic', delimiter=',')


2023-05-11 22:00:54,050 Reading data from /content/data_fst
2023-05-11 22:00:54,053 Train: /content/data_fst/train.csv
2023-05-11 22:00:54,054 Dev: None
2023-05-11 22:00:54,058 Test: /content/data_fst/test.csv


In [101]:
label_dict_csv = corpus_csv.make_label_dictionary(label_type = 'topic')


2023-05-11 22:00:58,738 Computing label dictionary. Progress:


2222it [00:15, 147.42it/s]

2023-05-11 22:01:13,819 Dictionary created for label 'topic' with 6 values: Purchase Order (seen 1021 times), General Inquiry (seen 679 times), Supplier Claim (seen 330 times), Repayment (seen 151 times), Supplier Fee (seen 41 times)


In [37]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=2837b5078bcff9b7c19a100cb371fcfc619135210851dc4c15e783c4d6c883d6
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [38]:
from flair.embeddings import SentenceTransformerDocumentEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path
from flair.embeddings import TransformerDocumentEmbeddings

In [102]:
# document_embeddings = TransformerDocumentEmbeddings('distilbert-base-uncased', fine_tune=True)

document_embeddings = SentenceTransformerDocumentEmbeddings('sentence-transformers/all-MiniLM-L6-v2')


In [103]:
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict_csv, multi_label=False, label_type="topic")


In [104]:
trainer = ModelTrainer(classifier, corpus_csv)


In [105]:
import torch
torch.cuda.empty_cache()

In [106]:
 trainer.train(data_folder,
              learning_rate=0.05,
              mini_batch_size=10,
              max_epochs=5)

2023-05-11 22:01:50,667 ----------------------------------------------------------------------------------------------------
2023-05-11 22:01:50,669 Model: "TextClassifier(
  (embeddings): SentenceTransformerDocumentEmbeddings(
    (model): SentenceTransformer(
      (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
      (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
      (2): Normalize()
    )
  )
  (decoder): Linear(in_features=384, out_features=6, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
  (locked_dropout): LockedDropout(p=0.0)
  (word_dropout): WordDropout(p=0.0)
  (loss_function): CrossEntropyLoss()
  (weights): None
  (weight_tensor) None
)"
2023-05-11 22:01:50,670 ----------------------------------------------------------------------------------------------------
2023-05

100%|██████████| 25/25 [00:04<00:00,  5.76it/s]

2023-05-11 22:02:36,853 Evaluating as a multi-label problem: False
2023-05-11 22:02:36,867 DEV : loss 1.0537689924240112 - f1-score (micro avg)  0.5789


2023-05-11 22:02:38,610 BAD EPOCHS (no improvement): 0
2023-05-11 22:02:38,612 saving best model
2023-05-11 22:02:38,839 ----------------------------------------------------------------------------------------------------
2023-05-11 22:02:43,670 epoch 2 - iter 22/223 - loss 1.01523541 - time (sec): 4.82 - samples/sec: 45.60 - lr: 0.050000
2023-05-11 22:02:47,495 epoch 2 - iter 44/223 - loss 1.03533480 - time (sec): 8.65 - samples/sec: 50.87 - lr: 0.050000
2023-05-11 22:02:51,254 epoch 2 - iter 66/223 - loss 1.02090602 - time (sec): 12.41 - samples/sec: 53.19 - lr: 0.050000
2023-05-11 22:02:55,801 epoch 2 - iter 88/223 - loss 1.01731645 - time (sec): 16.96 - samples/sec: 51.90 - lr: 0.050000
2023-05-11 22:02:59,994 epoch 2 - iter 110/223 - loss 1.01860884 - time (sec): 21.15 - samples/sec: 52.01 - lr: 0.050000
2023-05-11 22:03:03,243 epoch 2 - iter 132/223 - loss 1.01083909 - time (sec): 24.40 - samples/sec: 54.10 - lr: 0.050000
2023-05-11 22:03:07,077 epoch 2 - iter 154/223 - loss 1.00

100%|██████████| 25/25 [00:05<00:00,  4.24it/s]

...

2023-05-11 22:05:48,544 Evaluating as a multi-label problem: False
2023-05-11 22:05:48,565 DEV : loss 0.9074786901473999 - f1-score (micro avg)  0.6154


2023-05-11 22:05:51,192 BAD EPOCHS (no improvement): 2
2023-05-11 22:05:51,404 ----------------------------------------------------------------------------------------------------


100%|██████████| 44/44 [00:07<00:00,  6.09it/s]

2023-05-11 22:05:59,038 Evaluating as a multi-label problem: False
2023-05-11 22:05:59,052 0.617	0.617	0.617	0.617
2023-05-11 22:05:59,053 
Results:
- F-score (micro) 0.617
- F-score (macro) 0.4149
- Accuracy 0.617

By class:
                 precision    recall  f1-score   support

 Purchase Order     0.6336    0.8218    0.7155       202
General Inquiry     0.5812    0.5354    0.5574       127
 Supplier Claim     0.5800    0.4531    0.5088        64
      Repayment     0.8571    0.1765    0.2927        34
   Supplier Fee     0.0000    0.0000    0.0000         9

       accuracy                         0.6170       436
      macro avg     0.5304    0.3974    0.4149       436
   weighted avg     0.6148    0.6170    0.5914       436

2023-05-11 22:05:59,055 ----------------------------------------------------------------------------------------------------


{'test_score': 0.6169724770642202,
 'dev_score_history': [0.5789473684210527,
  0.6234817813765182,
  0.6396761133603239,
  0.631578947368421,
  0.6153846153846154],
 'train_loss_history': [1.1638455686002676,
  1.0026364984220475,
  0.9323894825204871,
  0.8879732504607749,
  0.8581971885061392],
 'dev_loss_history': [1.0537689924240112,
  0.982566237449646,
  0.9494995474815369,
  0.9231001138687134,
  0.9074786901473999]}

In [ ]:
new_clf = TextClassifier.load('/content/data_fst/best-model.pt')
